In [1]:
import requests
import bs4
from bs4 import BeautifulSoup as soup
import json
import pandas as pd
from pandas import DataFrame

import spacy
from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Span
nlp = spacy.load('en_core_web_sm')
from string import punctuation

def get_ld_json(url: str) -> dict:
    parser = "html.parser"
    req = requests.get(url)
    page = soup(req.text, parser)
    return json.loads("".join(page.find("script", {"type":"application/ld+json"}).contents))

url1 = "https://www.allrecipes.com/recipe/232549"
url2 = "https://www.allrecipes.com/recipe/267555/"
url3 = "https://www.allrecipes.com/recipe/78299/"
url4 = "https://www.allrecipes.com/recipe/13062/"

def get_ingredientScript(url):
    jsonld = get_ld_json(url)
    useful = jsonld[1]
    ingredients = useful["recipeIngredient"]
    return ingredients

In [24]:
def parse_ingredients(url):
    ingredients = get_ingredientScript(url)
    df = DataFrame (ingredients,columns=['ingredients'])
    # 2 cases
    # "number" ("()") ("unit") (adjective) "noun/subject - ingredient" (, other)
    # contains "to taste"
    df = df["ingredients"]
    df_taste = df[df.str.contains('to taste', case = False)]
    
    df_unit = df[~df.str.contains('to taste', case = False)]
    print(df_unit)
    
    # array of arrays: each array is amount, unit, ingredient, other
    ingredients_parsed = []
    
    for i in df_unit:
        doc = nlp(i)
        curr_arr = [0, 0, 0, 0]
        for token in doc:
            #print(token.text, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
            found_num = False
            # only get first number if it matches criteria
            if found_num == False and token.pos_ == "NOUN" and not token.is_alpha or token.pos_ == "NUM":
                curr_arr[0] = token.text
                found_num = True
            if token.dep_ == "ROOT":
                curr_arr[2] = token.text
                
            
        ingredients_parsed.append(curr_arr)
#             print(token.text, token.pos_, token.tag_, token.dep_,
#                 token.shape_, token.is_alpha, token.is_stop)
        #displacy.render(doc, style="dep") # change to serve when we go to python
    
    return ingredients_parsed

parse_ingredients(url1)

0                           ¼ cup butter
1      ½ small white onion, finely diced
2               3 small zucchinis, diced
3    3 ears corn, husks and silk removed
Name: ingredients, dtype: object
¼ NOUN NN compound ¼ False False
cup NOUN NN compound xxx True False
butter NOUN NN ROOT xxxx True False


½ NOUN NN nmod ½ False False
small ADJ JJ amod xxxx True False
white ADJ JJ amod xxxx True False
onion NOUN NN ROOT xxxx True False
, PUNCT , punct , False False
finely ADV RB advmod xxxx True False
diced VERB VBN acl xxxx True False


3 NUM CD nummod d False False
small ADJ JJ amod xxxx True False
zucchinis NOUN NNS ROOT xxxx True False
, PUNCT , punct , False False
diced VERB VBN acl xxxx True False


3 NUM CD nummod d False False
ears NOUN NNS compound xxxx True False
corn NOUN NN ROOT xxxx True False
, PUNCT , punct , False False
husks NOUN NNS appos xxxx True False
and CCONJ CC cc xxx True True
silk NOUN NN conj xxxx True False
removed VERB VBN ROOT xxxx True False


[['¼', 0, 0, 0], ['½', 0, 0, 0], ['3', 0, 0, 0], ['3', 0, 0, 0]]